CFD Campus

In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import requests

In [3]:
cfd_url="https://cfd.nu.edu.pk/all-departments/"
driver=webdriver.Chrome()
driver.get(cfd_url)
cfd_html=driver.page_source
driver.quit()
cfd_html

'<html lang="en-US"><head>\n<meta charset="UTF-8">\n<meta name="viewport" content="width=device-width, initial-scale=1">\n<link rel="profile" href="https://gmpg.org/xfn/11">\n\n<title>All Departments – FAST NUCES | CFD</title>\n<meta name="robots" content="max-image-preview:large">\n<link rel="dns-prefetch" href="//fonts.googleapis.com">\n<link rel="dns-prefetch" href="//s.w.org">\n<link rel="alternate" type="application/rss+xml" title="FAST NUCES | CFD » Feed" href="https://cfd.nu.edu.pk/feed/">\n<link rel="alternate" type="application/rss+xml" title="FAST NUCES | CFD » Comments Feed" href="https://cfd.nu.edu.pk/comments/feed/">\n\t\t<script src="https://embed.tawk.to/_s/v4/app/64e818b1586/js/twk-main.js" charset="UTF-8" crossorigin="*"></script><script src="https://embed.tawk.to/_s/v4/app/64e818b1586/js/twk-vendor.js" charset="UTF-8" crossorigin="*"></script><script src="https://embed.tawk.to/_s/v4/app/64e818b1586/js/twk-chunk-vendors.js" charset="UTF-8" crossorigin="*"></script><scr

In [4]:
soup_cfd=BeautifulSoup(cfd_html,"html.parser")
cfd_section = soup_cfd('section','kc-elm kc-css-249015 kc_row')
dept_url=[]
for ele in cfd_section:
    cfd_depts_links=ele.find_all('a')
    for url in cfd_depts_links:
        temp = url.get('href')
        if url:
            dept_url.append(temp)
dept_url 

['https://cfd.nu.edu.pk/department-cs',
 'https://cfd.nu.edu.pk/department-se',
 'https://cfd.nu.edu.pk/department-ee',
 'https://cfd.nu.edu.pk/department-sh',
 'https://cfd.nu.edu.pk/department-fsm']

In [5]:
depts_html=[]

for url in dept_url:
    driver=webdriver.Chrome()
    driver.get(url)
    temp_html=driver.page_source
    driver.quit()
    depts_html.append(temp_html)
len(depts_html)    

5

In [6]:
faculty_url=[]
hec_approved_lst=[]
for ele in depts_html:
    
    soup=BeautifulSoup(ele,'html.parser')
    faculty_html=soup.find_all('div','unitech-teacher__thumb')
    for ele in faculty_html:
        a_tags=ele.find_all('a')
        for url in a_tags:
            temp=url.get('href')
            faculty_url.append(temp)
    hec_approved_html=soup.find_all("div","unitech-teacher__hover__info")

    for ele in hec_approved_html:
        hec_approved_tag=ele.find_all('p','hec')
        if hec_approved_tag:
            hec_approved_lst.append(True)
        else:
            hec_approved_lst.append(False)
        
print("Total faculty in cfd : ",len(faculty_url))

Total faculty in cfd :  106


In [26]:
teacher_html=[]

for url in faculty_url:
    try:
        
        driver=webdriver.Chrome()
        driver.get(url)
        temp_html=driver.page_source
        driver.quit()
        teacher_html.append(temp_html)
    except:
        print("Error")
#len(teacher_html)   


Error


In [27]:
len (teacher_html)

105

In [70]:
id_lst = []
name_lst = []
imgURL_lst = []
designation_lst = []
dept_lst = []
email_lst = []
ext_lst = []
edu_lst = []

for url in teacher_html:
    tempSoup = BeautifulSoup(url, "html.parser")
    shortlink_tag = tempSoup.find_all('link', rel='shortlink')

    if shortlink_tag:
        shortlink_url = shortlink_tag[0].get('href')
        p_value = shortlink_url.split('=')[-1]
        id = p_value
    else:
        id = 0

    teacher_tag = tempSoup.find_all('div', 'teacher')
    name = teacher_tag[0].find('h2').text
    designation = teacher_tag[0].find('h6').text
    img = teacher_tag[0].find('img')
    imgURL = img.get('src')
    ext = 0
    teacher_address_tag = teacher_tag[0].find_all('ul', 'teacher__address')
    info = teacher_address_tag[0].find_all('li')
    dept = info[0].text.strip()
    email = info[1].text.strip()
    ext_text = info[-1].text.strip()
    ext_temp = ext_text.split(':')[-1].strip()
    if ext_temp:
        ext = ext_temp
    else:
        ext = 0
    education_html = tempSoup.find('div', class_='htc__skill__container progress__bar--2')
    if education_html:
        lines = education_html.text.split('\n')[1:]
        filtered_list = [item.strip() for item in lines if item.strip() != '']
        edu = filtered_list[1] if len(filtered_list) > 1 else ''
    else:
        edu = ''

    id_lst.append(id)
    name_lst.append(name)
    imgURL_lst.append(imgURL)
    designation_lst.append(designation)
    dept_lst.append(dept)
    email_lst.append(email)
    ext_lst.append(ext)
    edu_lst.append(edu)

print(len(edu_lst))


105


In [74]:
soup=BeautifulSoup(teacher_html[104], "html.parser")
html=soup.find('div',class_='htc__skill__container progress__bar--2').text
print(html)
lines = html.split('\n')[1:]
filtered_list = [item for item in lines if item != '']
filtered_list[1]


Education
M.S(Finance), Air University, Islamabad, Pakistan (2015)
M.Com(Finance), Islamia University of Bahawalpur, Pakistan (2010)
B.Com(Accounts), Islamia University of Bahawalpur, Pakistan (2007)



'M.S(Finance), Air University, Islamabad, Pakistan (2015)'

'ID'
'Name'
'Email'
'Department'
'Extension'
'Designation'
'Highest Education'
'image URL'
'hec_approved'






In [71]:
import pandas as pd
df = pd.DataFrame(list(zip(id_lst, name_lst,email_lst,dept_lst,ext_lst,designation_lst,edu_lst,imgURL_lst,hec_approved_lst)),
               columns =['ID','Name', 'Email','Department','Extension','Designation','Highest Education','image URL','hec_approved'])
df.to_csv('cfd.csv', index=False) 
df

,ID,Name,Email,Department,Extension,Designation,Highest Education,image URL,hec_approved
0,6074,Muhammad Ahmad,dr.ahmad@nu.edu.pk,Department of Computer Science,160,Associate Professor,Doctor of Philosophy (Hyperspectral Imaging) (...,https://cfd.nu.edu.pk/wp-content/uploads/2020/...,True
1,4748,Shahzad Sarfraz,shahzad.sarfraz@nu.edu.pk,Department of Computer Science,102,Professor,"Ph.D. in Remote Sensing Image Processing AIT, ...",https://cfd.nu.edu.pk/wp-content/uploads/2019/...,True
2,507933,Anwar Shah,Anwar.Shah@nu.edu.pk,Department of Computer Science,131,Assistant Professor,"PhD Computer Sciences, FAST-NUCES, Peshawar",https://cfd.nu.edu.pk/wp-content/uploads/2022/...,True
3,4752,Bilal Khan,khan.bilal@nu.edu.pk,Department of Computer Science,126,Assistant Professor,"Ph.D. (Computer Science & Engineering), Donggu...",https://cfd.nu.edu.pk/wp-content/uploads/2019/...,True
4,4772,Hashim Yasin,hashim.yasin@nu.edu.pk,Department of Computer Science,165,Assistant Professor,"Ph.D. (Computer Science), University of Bonn, ...",https://cfd.nu.edu.pk/wp-content/uploads/2019/...,True
...,...,...,...,...,...,...,...,...,...
100,4858,Umar Farooq,umar.farooq@nu.edu.pk,FAST School of Management,264,Assistant Professor,"PhD (Finance), COMSATS University Islamabad, L...",https://cfd.nu.edu.pk/wp-content/uploads/2019/...,False
101,508478,Ahmad Salah Ud Din,ahmad.salah@nu.edu.pk,FAST School of Management,256,Lecturer,"Chartered Financial Analyst (CFA), institute, ...",https://cfd.nu.edu.pk/wp-content/uploads/2023/...,True
102,508457,Amna Babar Tirmizey,amna.Tirmizey@nu.edu.pk,FAST School of Management,0,Lecturer,"MS, Innovation and Entrepreneurship, 2022",https://cfd.nu.edu.pk/wp-content/uploads/2023/...,False
103,4865,Hafiz Muhammad Zeeshan Raza,Zeeshan.raza@nu.edu.pk,FAST School of Management,265,Lecturer,"MBA, FAST National University of computer and ...",https://cfd.nu.edu.pk/wp-content/uploads/2019/...,False


Peshawar Campus

In [151]:

psw_links = ["http://pwr.nu.edu.pk/cs-faculty/", "http://pwr.nu.edu.pk/ee-faculty/", "http://pwr.nu.edu.pk/sh-faculty/"]
psw_html = []

for url in psw_links:
    try:
        r = requests.get(url)
        soup = BeautifulSoup(r.content, "html.parser")
        psw_html.append(soup)
    except Exception as e:
        print(e)
print(len(psw_html))

3


In [152]:
len(psw_html)

3

In [153]:
psw_faculty_links_html = []
for html in psw_html:
    psw_soup = html
    links_html = psw_soup.find_all('div', 'team-members')
    for html in links_html:
        a_tags = html.find_all('a','text-theme-color-2')
        for a_tag in a_tags:
            temp = a_tag.get('href')
            psw_faculty_links_html.append(temp)

len(psw_faculty_links_html)

41

In [154]:
psw_website='http://pwr.nu.edu.pk/'
psw_faculty_html=[]
for url in psw_faculty_links_html:
    try:
        r = requests.get(psw_website+url)
        psw_faculty_soup = BeautifulSoup(r.content, "html.parser")
        psw_faculty_html.append(psw_faculty_soup)
    except:
        print(f"Failed to fetch or parse HTML from URL: {url}")
len(psw_faculty_html)


41

In [155]:
psw_faculty_html[0]

<!DOCTYPE html>

<html dir="ltr" lang="en">
<head>
<!-- Meta Tags -->
<meta content="width=device-width,initial-scale=1.0" name="viewport"/>
<meta content="text/html; charset=utf-8" http-equiv="content-type">
<meta content="Fast - NUCES Peshawar | Campus Web Portal" name="description"/>
<meta content="computer science, software engineering, electrical engineering, cs, ee, phd, ms," name="keywords"/>
<meta content="mikaeel khalid" name="author"/>
<meta content="1053624431393436" property="fb:pages"/>
<!-- Page Title -->
<title>FAST - NUCES Peshawar</title>
<base href="/"/>
<!-- Favicon and Touch Icons -->
<link href="images/favicon.ico" rel="shortcut icon" type="image/ico"/>
<link href="#" rel="apple-touch-icon"/>
<link href="#" rel="apple-touch-icon" sizes="72x72"/>
<link href="#" rel="apple-touch-icon" sizes="114x114"/>
<link href="#" rel="apple-touch-icon" sizes="144x144"/>
<!-- Stylesheet -->
<link href="css/bootstrap.min.css" rel="stylesheet" type="text/css"/>
<link href="css/jquer

In [156]:
psw_id_lst=[]
psw_name_lst=[]
psw_designation_lst=[]
psw_imgURL_lst=[]
psw_education_lst=[]
psw_hec_lst=[]
psw_dept_lst=[]
psw_email_lst=[]
psw_ext_lst=[]
for i in range(0,41):
    psw_id = psw_faculty_links_html[i].split('=')[1]
    pswLinkHtml = psw_faculty_html[i]
    psw_name = pswLinkHtml.find("h4", "name").text
    psw_h5_tags=pswLinkHtml.find_all('h5')
    psw_hec_approved=True
    psw_designation=psw_h5_tags[2].get_text()
    psw_hec=psw_h5_tags[3].get_text()
    psw_education_html=pswLinkHtml.find("div", {"id":"education"})
    education_ = psw_education_html.find('li').text
    psw_contact_html=pswLinkHtml.find("div", {"id":"contact"})
    psw_contact_html_lst = psw_contact_html.find_all('li')
    psw_ext=psw_contact_html_lst[0].text
    psw_email=psw_contact_html_lst[1].text
    if psw_hec:
        psw_hec_approved=True
    else:
        psw_hec_approved=False
    psw_dept=psw_h5_tags[4].get_text()
    img_url_html=pswLinkHtml.find_all('div','thumb')
    img_tag_url=img_url_html[0].find_all('img')
    img_url=psw_website+img_tag_url[0].get('src').replace(" ","")
    psw_id_lst.append(psw_id)
    psw_name_lst.append(psw_name)
    psw_designation_lst.append(psw_designation)
    psw_imgURL_lst.append(img_url)
    psw_education_lst.append(education_)
    psw_hec_lst.append(psw_hec_approved)
    psw_dept_lst.append(psw_dept)
    psw_email_lst.append(psw_email)
    psw_ext_lst.append(psw_ext)

len(psw_id_lst)

41

In [157]:
import pandas as pd
df_psw = pd.DataFrame(list(zip(psw_id_lst, psw_name_lst,psw_email_lst,psw_dept_lst,psw_ext_lst,psw_designation_lst,psw_education_lst,psw_imgURL_lst,psw_hec_lst)),
               columns =['ID','Name', 'Email','Department','Extension','Designation','Highest Education','image URL','hec_approved'])
df_psw.to_csv('psw.csv', index=False) 
df_psw

,ID,Name,Email,Department,Extension,Designation,Highest Education,image URL,hec_approved
0,1,Dr. Hafeez Ur Rehman,hafeez.urrehman@nu.edu.pk,Computer Science,+92-334-112-8128 | Ext 107,Associate Professor & HoD,"Ph.D. in Computer and Control Engineering, Pol...",http://pwr.nu.edu.pk/images/faculty/dr_hafeez_...,True
1,2,Dr. Omar Usman Khan,omar.khan@nu.edu.pk,Computer Science,+92-334-112-8128 | Ext 103,Associate Professor & Director,"Ph.D. (Computer And Control Engineering), Poli...",http://pwr.nu.edu.pk/images/faculty/Dr_Omer_Us...,True
2,3,Dr. Nouman Azam,nouman.azam@nu.edu.pk,Computer Science,+92-334-112-8128 | Ext 134,Associate Professor,"Ph.D. (CS), University of Regina, Canada (2014)\r",http://pwr.nu.edu.pk/images/faculty/Dr_noman_a...,True
3,4,Dr. Mohammad Nauman,mohammad.nauman@nu.edu.pk,Computer Science,+92-334-112-8128 | Ext 126,Associate Professor,"PhD(IT), University of Kuala Lumpur, Malaysia ...",http://pwr.nu.edu.pk/images/faculty/Dr.noman2.jpg,True
4,5,Dr. Taimoor Khan,taimoor.khan@nu.edu.pk,Computer Science,+92-334-112-8128 | Ext 127,Assistant Professor,"Ph.D. (Computer Science), Bahria University Is...",http://pwr.nu.edu.pk/images/faculty/Dr_M_Taimo...,True
5,6,Dr. Bahar Ali,bahar.ali@nu.edu.pk,Computer Science,0911128128 | Ext 122,Assistant Professor,"PhD, Three-way Clustering (Machine Learning), ...",http://pwr.nu.edu.pk/images/faculty/dr_baharal...,True
6,7,Dr. Musadaq Mansoor,musadaq.mansoor@nu.edu.pk,Computer Science,+92-334-112-8128 | Ext 130,Assistant Professor,PhD Computer Science (Machine Learning / Bioin...,http://pwr.nu.edu.pk/images/faculty/dr_musadaq...,False
7,8,Dr. Muhammad Amin,muhammad.amin@nu.edu.pk,Computer Science,+92-334-112-8128 | Ext 146,Assistant Professor,"PhD (Computer Science), IM Sciences (2022)\r",http://pwr.nu.edu.pk/images/faculty/amin2.jpg,False
8,9,Fazl-e-Basit,fazl.basit@nu.edu.pk,Computer Science,+92-334-112-8128 | Ext 110,Assistant Professor,"M.S(Computer Science), NUST, Rawalpindi, Pakis...",http://pwr.nu.edu.pk/images/faculty/Fazl-e-Bas...,False
9,10,Shoaib Muhammad Khan,shoaib.khan@nu.edu.pk,Computer Science,+92-334-112-8128 | Ext 144,Assistant Professor,"M.S(Computer Science), NUCES, Islamabad, Pakis...",http://pwr.nu.edu.pk/images/faculty/shoaib_2.jpg,False


LAHORE CAMPUS

In [140]:
lhr_url="http://lhr.nu.edu.pk/faculty/"
try:
    r_url = requests.get(lhr_url)
    lhr_soup = BeautifulSoup(r_url.content, "html.parser")
except Exception as e:
    print(e)
lhr_soup

<!DOCTYPE html>

<html lang="en">
<head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1.0" name="viewport"/>
<meta content="ie=edge" http-equiv="X-UA-Compatible"/>
<link href="https://maxcdn.bootstrapcdn.com/bootstrap/4.2.1/css/bootstrap.min.css" rel="stylesheet"/>
<link href="https://cdnjs.cloudflare.com/ajax/libs/normalize/5.0.0/normalize.min.css" rel="stylesheet"/>
<link href="https://unpkg.com/aos@2.3.0/dist/aos.css" rel="stylesheet"/>
<link href="https://fonts.googleapis.com/css?family=Lato" rel="stylesheet"/>
<link href="https://fonts.googleapis.com/css?family=Lora" rel="stylesheet"/>
<link href="https://fonts.googleapis.com/css?family=Roboto" rel="stylesheet"/>
<link crossorigin="anonymous" href="https://use.fontawesome.com/releases/v5.7.1/css/all.css" integrity="sha384-fnmOCqbTlWIlj8LyTjo7mOUStjsKC4pOpQbqyi7RrhN7udi9RwhKkMHpvLbHG9Sr" rel="stylesheet"/>
<link href="/static/campus/CSS/stylesheet.css" rel="stylesheet" type="text/css">
<link href="/stati

In [141]:
f = lhr_soup.find_all('div', class_='facultyCard')
f

[<div class="card facultyCard card-br-0 ml-0 mr-0 pb-0">
 <div class="facultyImage">
 <img class="card-img-top rounded-circle mt-3 mb-0 d-block mx-auto" src="/media/Faculty/02_-_Dr._Kashif_Zafar_Prof__HoD_CS.png"/>
 </div>
 <div class="card-body pr-0 pl-0 pb-0">
 <h5 class="text-center">Dr. Kashif Zafar</h5>
 <p class="small text-center font-italic">Professor &amp; HOD
                         
                          <br> HEC Approved PhD Supervisor 
                         
                         </br></p>
 <p class="mb-0 text-center">kashif.zafar@nu.edu.pk</p>
 <!--  <p class="mt-0 small text-center">ext no: 221</p> -->
 <a class="faculty-link" href="/fsc/facultyProfile/1238"></a>
 </div>
 </div>,
 <div class="card facultyCard card-br-0 ml-0 mr-0 pb-0">
 <div class="facultyImage">
 <img class="card-img-top rounded-circle mt-3 mb-0 d-block mx-auto" src="/media/Faculty/03_-_Dr._Asif_Mahmood_Gilani_Professor_CS.JPG"/>
 </div>
 <div class="card-body pr-0 pl-0 pb-0">
 <h5 class="tex

In [143]:
lhr_id=[]
lhr_name=[]
lhr_email=[]
lhr_desig=[]
lhr_img=[]
lhr_hec_approved=[]
lhr_dept=[]

lhr_profile=[]
for i in range(len(f)):
    faculty = f[i]
    try:
        lhr_id.append(int(faculty.find('a',class_='faculty-link')['href'].split('/')[-1]))
    except:
        lhr_id.append(-1)
    lhr_name.append(faculty.find('h5', class_='text-center').text)
    lhr_desig.append(faculty.find('p',class_='small').text.strip().split('\n')[0])
    lhr_email.append(faculty.find('p', class_='mb-0').text)
    lhr_img.append('https://lhr.nu.edu.pk/'+faculty.find('img', class_='card-img-top')['src'])
    text = f[i].get_text()
    if 'HEC Approved PhD Supervisor' in text:
        lhr_hec_approved.append(True)
    else:
        lhr_hec_approved.append(False)
    lhr_dept.append(faculty.find('a',class_='faculty-link')['href'].split('/')[1])    
        
    lhr_profile.append('https://lhr.nu.edu.pk'+faculty.find('a',class_='faculty-link')['href'])


In [146]:
lhr_ext_lst=[]
lhr_education_lst=[]

for lhr_pro_link in lhr_profile:
    
    try:
        lhr_pro_link_soup = requests.get(lhr_pro_link)
        lhr_soup = BeautifulSoup(lhr_pro_link_soup.content, "html.parser")
    except Exception as e:
        print(e)

    lhr_soup_ext = lhr_soup.find_all('span', "small")

    if lhr_soup_ext:
        lhr_ext_faculty = lhr_soup_ext[0].text
        parts = lhr_ext_faculty.split("Ext:")
    
        if parts:
            ext_info = parts[1].strip()  
            lhr_ext_lst.append(ext_info)
        else:
            lhr_ext_lst.append(-1)
    edu_lhr_soup = BeautifulSoup(lhr_pro_link_soup.content, 'html.parser')
    edu_html_lhr = edu_lhr_soup.find_all('div', 'col-lg-8 col-md-6 col-sm-12 text-justify')
    if edu_html_lhr:
        first_li_tag = edu_html_lhr[0].find('li')
        if first_li_tag:
            edu_info_lhr = first_li_tag.text
            lhr_education_lst.append(edu_info_lhr)
        else:
            lhr_education_lst.append('None')
    else:
        lhr_education_lst.append('None')


len(lhr_education_lst)


184

In [147]:
for i in range(0,184):
    if lhr_dept[i] == 'cv':
        lhr_dept[i] = 'Civil Engineering'
    elif lhr_dept[i] == 'ee':
        lhr_dept[i] = 'Electrical Engineering'
    elif lhr_dept[i] == 'fsc':
        lhr_dept[i] = 'Computer Science'
    elif lhr_dept[i] == 'fsm':
        lhr_dept[i] = 'Fast School of Management'
    elif lhr_dept[i] == 'ss':
        lhr_dept[i] = 'Science and Humanities'
        

In [148]:
set(lhr_dept)

{'Civil Engineering',
 'Computer Science',
 'Electrical Engineering',
 'Fast School of Management',
 'Science and Humanities'}

In [158]:
import pandas as pd
df_lhr = pd.DataFrame(list(zip(lhr_id, lhr_name,lhr_email,lhr_dept,lhr_ext_lst,lhr_desig,lhr_education_lst,lhr_img,lhr_hec_approved)),
               columns =['ID','Name', 'Email','Department','Extension','Designation','Highest Education','image URL','hec_approved'])
df_lhr.to_csv('lhr.csv', index=False) 
df_lhr

,ID,Name,Email,Department,Extension,Designation,Highest Education,image URL,hec_approved
0,1238,Dr. Kashif Zafar,kashif.zafar@nu.edu.pk,Computer Science,221,Professor & HOD,"Ph.D(Computer Science), NUCES, Islamabad, Pa...",https://lhr.nu.edu.pk//media/Faculty/02_-_Dr._...,True
1,4391,Dr. Asif Mahmood Gillani,asif.gilani@nu.edu.pk,Computer Science,291,Professor,"Ph.D(Computing), University of Patras , Gree...",https://lhr.nu.edu.pk//media/Faculty/03_-_Dr._...,True
2,5424,Dr. Hammad Naveed,hammad.naveed@nu.edu.pk,Computer Science,227,Professor,"Ph.D. (Bioengineering), The University of Ill...",https://lhr.nu.edu.pk//media/Faculty/01_-_Dr._...,True
3,4027,Dr. Aamir Wali,aamir.wali@nu.edu.pk,Computer Science,260,Associate Professor,"PhD (CS), FAST-NUCES, Lahore, Pakistan (2019)",https://lhr.nu.edu.pk//media/Faculty/44_-_Dr._...,True
4,6174,Dr. Arshad Ali,arshad.ali1@nu.edu.pk,Computer Science,406,Associate Professor,"PhD (CS), University of Paris VI (UPMC) Pari...",https://lhr.nu.edu.pk//media/Faculty/04_-_Dr._...,True
...,...,...,...,...,...,...,...,...,...
179,6420,Ms. Nokhaiz Zahra,nokhaiz.zahra@nu.edu.pk,Science and Humanities,324,Instructor,"MA, Virtual University(2020)",https://lhr.nu.edu.pk//media/Faculty/165_-_Nok...,False
180,6181,Ms. Rida Ahmed,rida.ahmed@nu.edu.pk,Science and Humanities,363,Instructor,"BS (English), Fatima Jinnah Women University...",https://lhr.nu.edu.pk//media/Faculty/6181-remo...,False
181,6555,Ms. Aqsa Naz,aqsa.naz@nu.edu.pk,Science and Humanities,324,Lecturer (English),"M.Phil (English), UCP, Lahore (2022)",https://lhr.nu.edu.pk//media/Faculty/161_-_Aqs...,False
182,-1,Mr. Hafiz Umair Gulzar,umair.gulzar@nu.edu.pk,Science and Humanities,207,Lecturer (Islamiat),"M.Phil (Islamic Studies), GCU, Lahore (2019)",https://lhr.nu.edu.pk//media/Faculty/163_-_Haf...,False


In [159]:
df_cfd_data = pd.read_csv('cfd.csv')
df_cfd_data

,ID,Name,Email,Department,Extension,Designation,Highest Education,image URL,hec_approved
0,6074,Muhammad Ahmad,dr.ahmad@nu.edu.pk,Department of Computer Science,160,Associate Professor,Doctor of Philosophy (Hyperspectral Imaging) (...,https://cfd.nu.edu.pk/wp-content/uploads/2020/...,True
1,4748,Shahzad Sarfraz,shahzad.sarfraz@nu.edu.pk,Department of Computer Science,102,Professor,"Ph.D. in Remote Sensing Image Processing AIT, ...",https://cfd.nu.edu.pk/wp-content/uploads/2019/...,True
2,507933,Anwar Shah,Anwar.Shah@nu.edu.pk,Department of Computer Science,131,Assistant Professor,"PhD Computer Sciences, FAST-NUCES, Peshawar",https://cfd.nu.edu.pk/wp-content/uploads/2022/...,True
3,4752,Bilal Khan,khan.bilal@nu.edu.pk,Department of Computer Science,126,Assistant Professor,"Ph.D. (Computer Science & Engineering), Donggu...",https://cfd.nu.edu.pk/wp-content/uploads/2019/...,True
4,4772,Hashim Yasin,hashim.yasin@nu.edu.pk,Department of Computer Science,165,Assistant Professor,"Ph.D. (Computer Science), University of Bonn, ...",https://cfd.nu.edu.pk/wp-content/uploads/2019/...,True
...,...,...,...,...,...,...,...,...,...
100,4858,Umar Farooq,umar.farooq@nu.edu.pk,FAST School of Management,264,Assistant Professor,"PhD (Finance), COMSATS University Islamabad, L...",https://cfd.nu.edu.pk/wp-content/uploads/2019/...,False
101,508478,Ahmad Salah Ud Din,ahmad.salah@nu.edu.pk,FAST School of Management,256,Lecturer,"Chartered Financial Analyst (CFA), institute, ...",https://cfd.nu.edu.pk/wp-content/uploads/2023/...,True
102,508457,Amna Babar Tirmizey,amna.Tirmizey@nu.edu.pk,FAST School of Management,0,Lecturer,"MS, Innovation and Entrepreneurship, 2022",https://cfd.nu.edu.pk/wp-content/uploads/2023/...,False
103,4865,Hafiz Muhammad Zeeshan Raza,Zeeshan.raza@nu.edu.pk,FAST School of Management,265,Lecturer,"MBA, FAST National University of computer and ...",https://cfd.nu.edu.pk/wp-content/uploads/2019/...,False


Preprocessing andn Cleaning

In [160]:
df_cfd_data.loc[df_cfd_data['Department'] == 'Department of Computer Science', 'Department'] = 'Computer Science'
df_cfd_data.loc[df_cfd_data['Department'] == 'Department of Electrical Engineering', 'Department'] = 'Electrical Engineering'
df_cfd_data.loc[df_cfd_data['Department'] == 'Department of Software Engineering', 'Department'] = 'Software Engineering'
df_cfd_data.loc[df_cfd_data['Department'] == 'FAST School of Management', 'Department'] = 'Fast School of Management'
df_cfd_data.loc[df_cfd_data['Department'] == 'Department of Sciences & Humanities', 'Department'] = 'Science and Humanities'

In [161]:
df_cfd_data['Department'].unique()

array(['Computer Science', 'Software Engineering',
       'Electrical Engineering', 'Science and Humanities',
       'Fast School of Management'], dtype=object)

In [162]:
df_psw_data = pd.read_csv('psw.csv')
df_psw_data

,ID,Name,Email,Department,Extension,Designation,Highest Education,image URL,hec_approved
0,1,Dr. Hafeez Ur Rehman,hafeez.urrehman@nu.edu.pk,Computer Science,+92-334-112-8128 | Ext 107,Associate Professor & HoD,"Ph.D. in Computer and Control Engineering, Pol...",http://pwr.nu.edu.pk/images/faculty/dr_hafeez_...,True
1,2,Dr. Omar Usman Khan,omar.khan@nu.edu.pk,Computer Science,+92-334-112-8128 | Ext 103,Associate Professor & Director,"Ph.D. (Computer And Control Engineering), Poli...",http://pwr.nu.edu.pk/images/faculty/Dr_Omer_Us...,True
2,3,Dr. Nouman Azam,nouman.azam@nu.edu.pk,Computer Science,+92-334-112-8128 | Ext 134,Associate Professor,"Ph.D. (CS), University of Regina, Canada (2014)\r",http://pwr.nu.edu.pk/images/faculty/Dr_noman_a...,True
3,4,Dr. Mohammad Nauman,mohammad.nauman@nu.edu.pk,Computer Science,+92-334-112-8128 | Ext 126,Associate Professor,"PhD(IT), University of Kuala Lumpur, Malaysia ...",http://pwr.nu.edu.pk/images/faculty/Dr.noman2.jpg,True
4,5,Dr. Taimoor Khan,taimoor.khan@nu.edu.pk,Computer Science,+92-334-112-8128 | Ext 127,Assistant Professor,"Ph.D. (Computer Science), Bahria University Is...",http://pwr.nu.edu.pk/images/faculty/Dr_M_Taimo...,True
5,6,Dr. Bahar Ali,bahar.ali@nu.edu.pk,Computer Science,0911128128 | Ext 122,Assistant Professor,"PhD, Three-way Clustering (Machine Learning), ...",http://pwr.nu.edu.pk/images/faculty/dr_baharal...,True
6,7,Dr. Musadaq Mansoor,musadaq.mansoor@nu.edu.pk,Computer Science,+92-334-112-8128 | Ext 130,Assistant Professor,PhD Computer Science (Machine Learning / Bioin...,http://pwr.nu.edu.pk/images/faculty/dr_musadaq...,False
7,8,Dr. Muhammad Amin,muhammad.amin@nu.edu.pk,Computer Science,+92-334-112-8128 | Ext 146,Assistant Professor,"PhD (Computer Science), IM Sciences (2022)\r",http://pwr.nu.edu.pk/images/faculty/amin2.jpg,False
8,9,Fazl-e-Basit,fazl.basit@nu.edu.pk,Computer Science,+92-334-112-8128 | Ext 110,Assistant Professor,"M.S(Computer Science), NUST, Rawalpindi, Pakis...",http://pwr.nu.edu.pk/images/faculty/Fazl-e-Bas...,False
9,10,Shoaib Muhammad Khan,shoaib.khan@nu.edu.pk,Computer Science,+92-334-112-8128 | Ext 144,Assistant Professor,"M.S(Computer Science), NUCES, Islamabad, Pakis...",http://pwr.nu.edu.pk/images/faculty/shoaib_2.jpg,False


In [ ]:
df_psw_data['Extension'] = df_psw_data['Extension'].str.extract(r'Ext (\d+)')
df_psw_data['Extension']

In [ ]:
df_psw_data.to_csv('psw.csv', index=False)
df_psw_data.info()

In [185]:
df_lhr["Extension"].replace('None', -1, inplace=True)

In [188]:
df_psw_data["Extension"] = df_psw_data["Extension"].astype("int")
df_cfd_data["Extension"] = df_cfd_data["Extension"].astype("int")
df_lhr["Extension"] = df_lhr["Extension"].astype("int")

In [216]:
df_lhr.to_csv('lhr.csv', index=False)
df_cfd_data.to_csv('cfd.csv', index=False)
df_psw_data.to_csv('psw.csv', index=False)

Concatenation of Dataframes

In [227]:
concatednated_df=pd.concat([df_lhr,df_cfd_data,df_psw_data],ignore_index=True)
concatednated_df

,ID,Name,Email,Department,Extension,Designation,Highest Education,image URL,hec_approved
0,1238,Dr. Kashif Zafar,kashif.zafar@nu.edu.pk,Computer Science,221,Professor & HOD,"Ph.D(Computer Science), NUCES, Islamabad, Pa...",https://lhr.nu.edu.pk//media/Faculty/02_-_Dr._...,True
1,4391,Dr. Asif Mahmood Gillani,asif.gilani@nu.edu.pk,Computer Science,291,Professor,"Ph.D(Computing), University of Patras , Gree...",https://lhr.nu.edu.pk//media/Faculty/03_-_Dr._...,True
2,5424,Dr. Hammad Naveed,hammad.naveed@nu.edu.pk,Computer Science,227,Professor,"Ph.D. (Bioengineering), The University of Ill...",https://lhr.nu.edu.pk//media/Faculty/01_-_Dr._...,True
3,4027,Dr. Aamir Wali,aamir.wali@nu.edu.pk,Computer Science,260,Associate Professor,"PhD (CS), FAST-NUCES, Lahore, Pakistan (2019)",https://lhr.nu.edu.pk//media/Faculty/44_-_Dr._...,True
4,6174,Dr. Arshad Ali,arshad.ali1@nu.edu.pk,Computer Science,406,Associate Professor,"PhD (CS), University of Paris VI (UPMC) Pari...",https://lhr.nu.edu.pk//media/Faculty/04_-_Dr._...,True
...,...,...,...,...,...,...,...,...,...
325,54,Mr. Osama Sohrab,osama.sohrab@nu.edu.pk,Science and Humanities,144,Lecturer,"M.S (Applied Mathematics), Ghulam Ishaq Khan I...",http://pwr.nu.edu.pk/images/faculty/Osama_Sohr...,False
326,53,Mr. Ikram Ullah,ikram.ullah@nu.edu.pk,Science and Humanities,145,Assistant Professor,#,http://pwr.nu.edu.pk/images/faculty/Ikram_Ulla...,False
327,52,Mr. Askar Ali,askar.ali@nu.edu.pk,Science and Humanities,145,Lecturer,#,http://pwr.nu.edu.pk/images/faculty/Askar_Ali2...,False
328,51,Ms. Noreen Shah,noreen.shah@nu.edu.pk,Science and Humanities,150,Lecturer,Ph.D. Scholar (Coursework completed),http://pwr.nu.edu.pk/images/faculty/WhatsAppIm...,False


In [229]:
concatednated_df.to_csv("fast_faculty.csv",index=False)

In [235]:
faculty=pd.read_csv('fast_faculty.csv')
frac_val=9/10
print(frac_val)
sample_=faculty.sample(frac=frac_val)
sample_.to_csv('sample.csv',index=False)

0.9


In [236]:
sample_df=pd.read_csv('sample.csv')
sample_df

,ID,Name,Email,Department,Extension,Designation,Highest Education,image URL,hec_approved
0,9172,Dr. Hira Iqbal,hira.iqbal@nu.edu.pk,Science and Humanities,347,Assistant Professor,"PhD (Mathematics), National University of Co...",https://lhr.nu.edu.pk//media/Faculty/9172.jpg,True
1,6720,Ms. Amina Tabassum,amina.tabassum@nu.edu.pk,Fast School of Management,-1,Lecturer,"MS, Northeastern University, USA (2023)",https://lhr.nu.edu.pk//media/Faculty/6720.png,False
2,6553,Mr. Danyal Farhat,danyal.farhat@nu.edu.pk,Computer Science,520,Lecturer,"MS (CS), UCP, Lahore (2021)",https://lhr.nu.edu.pk//media/Faculty/6553.jpg,False
3,6286,Muhammad Iqbal,iqbal.m@nu.edu.pk,Science and Humanities,282,Lecturer,"M.Phil. (Linguistics), The Islamia University ...",https://cfd.nu.edu.pk/wp-content/uploads/2021/...,False
4,508274,Muhammad Munawar,m.munawar@nu.edu.pk,Science and Humanities,283,Lecturer,"MS(Math), COMSATS University, Islamabad, 2015",https://cfd.nu.edu.pk/wp-content/uploads/2023/...,False
...,...,...,...,...,...,...,...,...,...
292,3720,Mr. Aamir Raheem,aamir.raheem@nu.edu.pk,Computer Science,257,Assistant Professor,"M.S(Computer Science), NUCES,Lahore, Pakista...",https://lhr.nu.edu.pk//media/Faculty/18-_Aamir...,False
293,508158,Farwa Ahmad,farwa.ahmad@nu.edu.pk,Software Engineering,572,Instructor,MS (CS) National University of Computer and Em...,https://cfd.nu.edu.pk/wp-content/uploads/2023/...,False
294,5114,Dr. Mirza Mubasher Baig,mubasher.baig@nu.edu.pk,Computer Science,439,Assistant Professor,"Ph.D(Computer Science), Lahore University of...",https://lhr.nu.edu.pk//media/Faculty/46-_Dr._M...,True
295,506552,Haseeb Arshad,haseeb.arshad@nu.edu.pk,Computer Science,149,Lecturer,"Masters in Computer Science, 2020",https://cfd.nu.edu.pk/wp-content/uploads/2021/...,False
